# ONEAudit Demo

ONEAudit (Refereed paper: https://link.springer.com/chapter/10.1007/978-3-031-48806-1_5 Full version: https://arxiv.org/abs/2303.03335 ) is a way to use batch tally information efficiently in RLAs. 
It is vastly more efficient than batch-level comparison auditing.
For each SHANGRLA assertion, ONEAudit creates an "average" assorter value for each reporting batch.
That is then used in a ballot-level comparison audit based on comparing the assorter applied to the manually ascertained vote (MVR, manual vote record) on each ballot card the audit selects to the average assorter value for the batch to which each ballot belongs.

ONEAudit is useful in a variety of situations, including:

+ when batch tallies are available, whether or not the batches correspond to physical batches, in contrast to traditional batch-level comparison audits, which work only when reporting batches correspond to physical batches
+ when CVRs are available for batches of cards but there is no mapping from individual cards to individual CVRs (this is common for precinct-count optical scan systems)
+ when CVRs are available for some cards but not others (ONEAudit improves on SUITE in that situation, avoiding the need for stratification)

`SHANGRLA` currently infers batch information from Dominion CVRs as currently (that is, as of August 2024) used by San Francisco, as follows:

+ when reading the CVRs using `shangrla.formats.Dominion.read_cvrs()`, set `pool_groups` to be the list of `CountingGroupID`s that should be audited using ONEAudit CVRs derived from batch tallies.
    - For SF, that is `CountingGroupID == 1`.
    - if `pool_groups` is nonempty, the CVRs with `CountingGroupID in pool_groups` are marked as `pool`
+ apply `shangrla.core.Audit.CVR.check_tally_pools()`: ensure every CVR in each `tally_pool` has the same value of `pool`
+ apply`shangrla.core.Audit.CVR.add_pool_contests()`: ensure every CVR in each `tally_pool` for which `pool == True` has every contest in the tally_pool
+ create the assertions for the contests under audit using functions in `shangrla.core.Audit.Assertion`
+ apply `shangrla.core.Audit.Assorter.set_tally_pool_means()` to set the assorter means
+ estimate initial sample size
+ audit then proceeds as a standard ballot-level comparison audit. The MVR for each inspected card will automatically be compared to the ONEAudit CVRs for the batch to which that card belongs


## Workflow for a ONEAudit RLA

+ Read overall audit information (including the seed) and contest information
+ Read RAIRE assertions for IRV contests and construct assertions for all other contests
+ Read ballot manifest
+ For comparison audits, read cvrs. Every CVR should have a corresponding manifest entry.
+ Prepare ~2EZ:
    - `N_phantoms = max_cards - cards_in_manifest`
    - If `N_phantoms < 0`: complain
    - Else: create `N_phantoms` phantom cards
    - For each contest `c`:
        + `N_c` is the input upper bound on the number of cards that contain `c`
        + If `N_c is None`: `N_c = max_cards - non_c_cvrs`, where `non_c_cvrs` is #CVRs that don't contain `c`
        + `C_c` is the number of CVRs that contain the contest
        + If `C_c > N_c`: complain
        + Else if `N_c - C_c > N_phantoms`: complain
        + Else:
            - Consider contest `c` to be on the first `N_c - C_c` phantom CVRs
            - Consider contest `c` to be on the first `N_c - C_c` phantom ballots
+ Create Assertions for every Contest. This involves also creating an Assorter for every Assertion, and a `NonnegMean` test
for every Assertion.
+ Calculate assorter margins for all assorters:
    - If `not use_style`: apply the Assorter to all cards and CVRs, including phantoms
    - Else: apply the assorter only to cards/cvrs reported to contain the contest, including phantoms that contain the contest
+ Set `assertion.test.u` to the appropriate value for each assertion: `assorter.upper_bound` for polling audits or 
      `2/(2-assorter.margin/assorter.upper_bound)` for ballot-level comparison audits
+ Estimate starting sample size for the specified sampling design (w/ or w/o replacement, stratified, etc.), for chosen risk function, use of card-style information, etc.:
    - User-specified criterion, controlled by parameters. Examples:
        + expected sample size for completion, on the assumption that there are no errors
        + 90th percentile of sample size for completion, on the assumption that errors are not more frequent than specified
    - If `not use_style`: base estimate on sampling from the entire manifest, i.e., smallest assorter margin
    - Else: use consistent sampling:
        + Augment each CVR (including phantoms) with a probability of selection, `p`, initially 0
        + For each contest `c`:
            - Find sample size `n_c` that meets the criterion 
            - For each non-phantom CVR that contains the contest, set `p = max(p, n_c/N_c)` 
        + Estimated sample size is the sum of `p` over all non-phantom CVRs
+ Draw the random sample:
    - Use the specified design, including using consistent sampling for style information
    - Express sample cards in terms of the manifest
    - Export
+ Read manual interpretations of the cards (MVRs)
+ Calculate attained risk for each assorter
    - Use ~2EZ to deal with phantom CVRs or cards; the treatment depends on whether `use_style == True`
+ Report
+ Estimate incremental sample size if any assorter nulls have not been rejected
+ Draw incremental sample; etc

# Audit parameters.

The overall audit involves information that is the same across contests, encapsulated in
a dict called `audit`:

* `seed`: the numeric seed for the pseudo-random number generator used to draw sample (for SHA256 PRNG)
* `sim_seed`: seed for simulations to estimate sample sizes (for Mersenne Twister PRNG)
* `quantile`: quantile of the sample size to use for setting initial sample size
* `cvr_file`: filename for CVRs (input)
* `manifest_file`: filename for ballot manifest (input)
* `use_style`: Boolean. If True, use card style information (inferred from CVRs) to target samples. If False, sample from all cards, regardless of the contest.
* `sample_file`: filename for sampled card identifiers (output)
* `mvr_file`: filename for manually ascertained votes from sampled cards (input)
* `log_file`: filename for audit log (output)
* `error_rate_1`: expected rate of 1-vote overstatements. Recommended value $\ge$ 0.001 if there are hand-marked ballots. Larger values increase the initial sample size, but make it more likely that the audit will conclude after a single round even if the audit finds errors
* `error_rate_2`: expected rate of 2-vote overstatements. 2-vote overstatements should be extremely rare.
Recommended value: 0. Larger values increase the initial sample size, but make it more likely that the audit will conclude after a single round even if the audit finds errors
* `reps`: number of replications to use to estimate sample sizes. If `reps is None`, uses a deterministic method
* `quantile`: quantile of sample size to estimate. Not used if `reps is None`
* `strata`: a dict describing the strata. Keys are stratum identifiers; values are dicts containing:
    + `max_cards`: an upper bound on the number of pieces of paper cast in the contest. This should be derived independently of the voting system. A ballot consists of one or more cards.
    + `replacement`: whether to sample from this stratum with replacement. 
    + `use_style`: True if the sample in that stratum uses card-style information.
    + `audit_type` one of Contest.POLLING, Contest.CARD_COMPARISON, Contest.BATCH_COMPARISON but only POLLING and CARD_COMPARISON are currently implemented. 
    + `test`: the name of the function to be used to measure risk. Options are `kaplan_markov`,`kaplan_wald`,`kaplan_kolmogorov`,`wald_sprt`,`kaplan_mart`, `alpha_mart`, `betting_mart`. 
Not all risk functions work with every social choice function or every sampling method. 
    + `estim`: the estimator to be used by the `alpha_mart` risk function. Options:  
        - `fixed_alternative_mean` (default)
        - `shrink_trunc`
        - `optimal_comparison`
    + `bet`: the method to select the bet for the `betting_mart` risk function. Options:
        - `fixed_bet` (default)
        - `agrapa`
    + `test_kwargs`: keyword arguments for the risk function

----

* `contests`: a dict of contest-specific data 
    + the keys are unique contest identifiers for contests under audit
    + the values are Contest objects with attributes:
        - `risk_limit`: the risk limit for the audit of this contest
        - `cards`: an upper bound on the number of cast cards that contain the contest
        - `choice_function`: `Audit.SOCIAL_CHOICE_FUNCTION.PLURALITY`, 
          `Audit.SOCIAL_CHOICE_FUNCTION.SUPERMAJORITY`, or `Audit.SOCIAL_CHOICE_FUNCTION.IRV`
        - `n_winners`: number of winners for majority contests. (Multi-winner IRV, aka STV, is not supported)
        - `share_to_win`: for super-majority contests, the fraction of valid votes required to win, e.g., 2/3.
           (share_to_win*n_winners must be less than 100%)
        - `candidates`: list of names or identifiers of candidates
        - `reported_winners` : list of identifier(s) of candidate(s) reported to have won.
           Length should equal `n_winners`.
        - `assertion_file`: filename for a set of json descriptors of Assertions (see technical documentation) that collectively imply the reported outcome of the contest is correct. Required for IRV; ignored for other social choice functions
        - `audit_type`: the audit strategy. Currently `Audit.AUDIT_TYPE.POLLING (ballot-polling)`, 
           `Audit.AUDIT_TYPE.CARD_COMPARISON` (ballot-level comparison audits), and `Audit.AUDIT_TYPE.ONEAUDIT`
            are implemented. HYBRID and STRATIFIED are planned.
        - `test`: the risk function for the audit. Default is `NonnegMean.alpha_mart`, the alpha supermartingale test
        - `estim`: estimator for the alternative hypothesis for the test. Default is `NonnegMean.shrink_trunc`
        - `use_style`: True to use style information from CVRs to target the sample. False for polling audits or for sampling from all ballots for every contest.
        - other keys and values are added by the software, including `cvrs`, the number of CVRs that contain the contest, and `p`, the sampling fraction expected to be required to confirm the contest

In [1]:
# if shangrla has not already been installed, install it then restart the kernel
# !pip install -e "../"

In [5]:
import math
import json
import warnings
import numpy as np
import pandas as pd
import csv
import copy

import glob
import os, sys

from collections import OrderedDict
from IPython.display import display, HTML

from cryptorandom.cryptorandom import SHA256, int_from_hash
from cryptorandom.sample import sample_by_index

from shangrla.core.Audit import Audit, Assertion, Assorter, Contest, CVR, Stratum
from shangrla.core.NonnegMean import NonnegMean
from shangrla.formats.Dominion import Dominion

sys.path.append(os.path.realpath('./SHANGRLA'))

In [6]:
audit = Audit.from_dict({
         'seed':           12345678901234567890,
         'sim_seed':       314159265,
         'cvr_file':       './data/SF_CVR_Export_20240311150227/CvrExport_*.json',
         'manifest_file':  './data/SF_CVR_Export_20240311150227/dummy.xlsx',
         'sample_file':    './data/sample.csv',
         'mvr_file':       './data/mvr.json',
         'log_file':       './data/log.json',
         'quantile':       0.8,
         'error_rate_1':   0.001,
         'error_rate_2':   0.0,
         'reps':           100,
         'strata':         {'stratum_1': {'max_cards':   293555, 
                                          'use_style':   True,
                                          'replacement': False,
                                          'audit_type':  Audit.AUDIT_TYPE.CARD_COMPARISON,
                                          'test':        NonnegMean.alpha_mart,
                                          'estimator':   NonnegMean.optimal_comparison,
                                          'test_kwargs': {}
                                         }
                           }
        })



In [7]:
# find upper bound on total cards across strata
audit.max_cards = np.sum([s.max_cards for s in audit.strata.values()])
'''
max_cards = 293555 # 146662 VBM turnout per SF Elections release 12 
https://sfelections.sfgov.org/november-5-2019-election-results-summary
'''

'\nmax_cards = 293555 # 146662 VBM turnout per SF Elections release 12 \nhttps://sfelections.sfgov.org/november-5-2019-election-results-summary\n'

In [8]:
# TO DO
# extract contests and candidates from relevant manifests
# ingest a list of contests to audit; include max cards cast in each contest or a flag to use the CVRs as an approximation
# find winner(s) of the audited contests from CVRs
# set "boilerplate" variables: risk limits, cards, audit_type, etc.
# construct contest dict

In [9]:
# contests to audit. Edit with details of your contest (eg., Contest 339 is the DA race)
contest_dict = {'1':{
                   'name': 'PRESIDENT OF THE UNITED STATES-DEM',
                   'risk_limit':       0.05,
                   'cards':            443578,
                   'choice_function':  Contest.SOCIAL_CHOICE_FUNCTION.PLURALITY,
                   'n_winners':        1,
                   'candidates':       ['5',],
                   'winner':           ['5'],
                   'assertion_file':   None,
                   'audit_type':       Audit.AUDIT_TYPE.CARD_COMPARISON,
                   'test':             NonnegMean.alpha_mart,
                   'estim':            NonnegMean.optimal_comparison
                  },
               '2':{
                   'name': 'PRESIDENT OF THE UNITED STATES-REP',
                   'risk_limit':       0.05,
                   'cards':            443578,
                   'choice_function':  Contest.SOCIAL_CHOICE_FUNCTION.PLURALITY,
                   'n_winners':        1,
                   'candidates':       ['9','10','11','12'],
                   'winner':           ['11'],
                   'assertion_file':   None,
                   'audit_type':       Audit.AUDIT_TYPE.CARD_COMPARISON,
                   'test':             NonnegMean.alpha_mart,
                   'estim':            NonnegMean.optimal_comparison
                  }
               }

contests = Contest.from_dict_of_dicts(contest_dict)

Example of other social choice functions:

        contests =  {'city_council':{'name': 'City Council',
                             'risk_limit':0.05,
                             'cards': None,
                             'choice_function': Contest.SOCIAL_CHOICE_FUNCTION.PLURALITY,
                             'n_winners':3,
                             'candidates':['Doug','Emily','Frank','Gail','Harry'],
                             'winner' : ['Doug', 'Emily', 'Frank']
                            },
                        'measure_1':{'name': 'Measure 1',
                             'risk_limit':0.05,
                             'cards': 65432,
                             'choice_function': Contest.SOCIAL_CHOICE_FUNCTION.SUPERMAJORITY,
                             'share_to_win':2/3,
                             'n_winners':1,
                             'candidates':['yes','no'],
                             'winner' : ['yes']
                            }                  
                      }

In [10]:
# read the assertions for the IRV contest
for c in contests:
    if contests[c].choice_function == Contest.SOCIAL_CHOICE_FUNCTION.IRV:
        with open(contests[c].assertion_file, 'r') as f:
            contests[c].assertion_json = json.load(f)['audits'][0]['assertions']

In [11]:
# construct the dict of dicts of assertions for each contest
Assertion.make_all_assertions(contests)

True

In [12]:
audit.check_audit_parameters(contests)

## Read the ballot manifest

In [13]:
# special for Primary/Dominion manifest format
manifest = pd.read_excel(audit.manifest_file)

FileNotFoundError: [Errno 2] No such file or directory: './data/SF_CVR_Export_20240311150227/dummy.xlsx'

## Read the CVR data and create CVR objects

In [14]:
# for ballot-level comparison audits
cvr_list = []
for _fname in glob.glob(audit.cvr_file):
    cvr_list.extend(Dominion.read_cvrs(_fname, use_current=True, enforce_rules=True, include_groups=(1,2)))
    
cvr_list = Dominion.raire_to_dominion(cvr_list)

In [15]:
len(cvr_list)

443578

In [16]:
# double-check whether the manifest accounts for every card
audit.max_cards, np.sum(manifest['Total Ballots'])

NameError: name 'manifest' is not defined

In [41]:
# Check that there is a card in the manifest for every card (possibly) cast. If not, add phantoms.
manifest, manifest_cards, phantom_cards = Dominion.prep_manifest(manifest, audit.max_cards, len(cvr_list))
manifest

,Tray #,Tabulator Number,Batch Number,Total Ballots,VBMCart.Cart number,cum_cards
0,1,99808,78,116,3,116
1,1,99808,77,115,3,231
2,1,99808,79,120,3,351
3,1,99808,81,76,3,427
4,1,99808,80,116,3,543
...,...,...,...,...,...,...
5476,3506,99815,86,2,19,292557
5477,3506,99815,84,222,19,292779
5478,3506,99815,83,346,19,293125
5479,3506,99815,82,332,19,293457


## Create CVRs for phantom cards

In [42]:
# For Comparison Audits Only
#----------------------------

# If the sample draws a phantom card, these CVRs will be used in the comparison.
# phantom MVRs should be treated as zeros by the Assorter for every contest

# setting use_style = False to generate phantoms

cvr_list, phantom_vrs = CVR.make_phantoms(audit=audit, contests=contests, cvr_list=cvr_list, prefix='phantom-1-')
print(f"Created {phantom_vrs} phantom records")

Created 0 phantom records


In [43]:
# find the mean of the assorters for the CVRs and check whether the assertions are met
min_margin = Assertion.set_all_margins_from_cvrs(audit=audit, contests=contests, cvr_list=cvr_list)

print(f'minimum assorter margin {min_margin}')
Contest.print_margins(contests)

minimum assorter margin 0.019902906001554532
margins in contest 339:
	assertion 18 v 17 elim 15 16 45: 0.045792366120740224
	assertion 17 v 16 elim 15 18 45: 0.019902906001554532
	assertion 15 v 18 elim 16 17 45: 0.028923647570604505
	assertion 18 v 16 elim 15 17 45: 0.0830003681935334
	assertion 17 v 16 elim 15 45: 0.058079120699294995
	assertion 15 v 17 elim 16 45: 0.08064120222007065
	assertion 15 v 17 elim 16 18 45: 0.10951712099930444
	assertion 18 v 16 elim 15 45: 0.14875018750596603
	assertion 15 v 16 elim 17 45: 0.13548158350492967
	assertion 15 v 16 elim 17 18 45: 0.1365247985163165
	assertion 15 v 16 elim 18 45: 0.16666893946625572
	assertion 15 v 16 elim 45: 0.15626406294745743
	assertion 15 v 45: 0.2956457705472446


In [44]:
audit.write_audit_parameters(contests=contests)

## Set up for sampling

## Find initial sample size

In [45]:
# find initial sample size 
sample_size = audit.find_sample_size(contests, cvrs=cvr_list)  
print(f'{sample_size=}\n{[(i, c.sample_size) for i, c in contests.items()]}')

sample_size=372
[('339', 372)]


## Draw the first sample

In [46]:
# draw the initial sample using consistent sampling
prng = SHA256(audit.seed)
CVR.assign_sample_nums(cvr_list, prng)

True

In [47]:
sampled_cvr_indices = CVR.consistent_sampling(cvr_list=cvr_list, contests=contests)
n_sampled_phantoms = np.sum(sampled_cvr_indices > manifest_cards)
print(f'The sample includes {n_sampled_phantoms} phantom cards.')

The sample includes 0 phantom cards.


In [48]:
len(cvr_list), manifest_cards, audit.max_cards

(146662, 293555, 293555)

In [49]:
# for comparison audit
cards_to_retrieve, sample_order, cvr_sample, mvr_phantoms_sample = \
    Dominion.sample_from_cvrs(cvr_list, manifest, sampled_cvr_indices)

# for polling audit
# cards_to_retrieve, sample_order, mvr_phantoms_sample = Dominion.sample_from_manifest(manifest, sample)

In [50]:
# write the sample
Dominion.write_cards_sampled(audit.sample_file, cards_to_retrieve, print_phantoms=False)

## Read the audited sample data

In [51]:
# for real data
with open(audit.mvr_file) as f:
    mvr_json = json.load(f)

mvr_sample = CVR.from_dict(mvr_json['ballots'])

# for simulated data, no errors
mvr_sample = cvr_sample.copy()

## Find measured risks for all assertions

In [52]:
CVR.prep_comparison_sample(mvr_sample, cvr_sample, sample_order)  # for comparison audit
# CVR.prep_polling_sample(mvr_sample, sample_order)  # for polling audit

###### TEST
# permute part of the sample to introduce errors deliberately
mvr_sample = cvr_sample.copy()
n_errs = 5
errs = mvr_sample[0:n_errs].copy()
np.random.seed(12345678)
np.random.shuffle(errs)
mvr_sample[0:n_errs] = errs

In [53]:
p_max = Assertion.set_p_values(contests=contests, mvr_sample=mvr_sample, cvr_sample=cvr_sample)
print(f'maximum assertion p-value {p_max}')
done = audit.summarize_status(contests)

maximum assertion p-value 0.48779308744628547

p-values for assertions in contest 339
	18 v 17 elim 15 16 45: 0.47847909464463045
	17 v 16 elim 15 18 45: 0.48779308744628547
	15 v 18 elim 16 17 45: 0.44631352397209006
	18 v 16 elim 15 17 45: 2.877844184074244e-07
	17 v 16 elim 15 45: 0.0035354717267599813
	15 v 17 elim 16 45: 4.0458461637126434e-07
	15 v 17 elim 16 18 45: 2.3358046805505937e-07
	18 v 16 elim 15 45: 5.790407277111407e-13
	15 v 16 elim 17 45: 9.312863307985585e-12
	15 v 16 elim 17 18 45: 1.8495628594508403e-09
	15 v 16 elim 18 45: 1.1604860427653792e-14
	15 v 16 elim 45: 9.55405920633293e-14
	15 v 45: 1.373326785425354e-26

contest 339 audit INCOMPLETE at risk limit 0.05. Attained risk 0.48779308744628547
assertions remaining to be proved:
	18 v 17 elim 15 16 45	contest_id: 339 winner: 18 loser: 17 assorter: contest_id: 339
upper bound: 1, winner defined: False, loser defined: False, assort defined: Truetally_pool_means: None p-value: 0.47847909464463045 margin: 0.045792

In [54]:
# Log the status of the audit 
audit.write_audit_parameters(contests)

# How many more cards should be audited?

Estimate how many more cards will need to be audited to confirm any remaining contests. The enlarged sample size is based on:

* cards already sampled
* the assumption that we will continue to see errors at the same rate observed in the sample

In [55]:
# Estimate sample size required to confirm the outcome, if errors continue
# at the same rate as already observed.

new_size = audit.find_sample_size(contests, cvrs=cvr_list, mvr_sample=mvr_sample, cvr_sample=cvr_sample)
print(f'{new_size=}\n{[(i, c.sample_size) for i, c in contests.items()]}')


new_size=7098
[('339', 6726)]


In [56]:
# save the first sample
sampled_cvr_indices_old, cards_to_retrieve_old, sample_order_old, cvr_sample_old, mvr_phantoms_sample_old = \
    sampled_cvr_indices, cards_to_retrieve,     sample_order,     cvr_sample,     mvr_phantoms_sample

In [57]:
# draw the sample
sampled_cvr_indices = CVR.consistent_sampling(cvr_list=cvr_list, contests=contests)
n_sampled_phantoms = np.sum(sampled_cvr_indices > manifest_cards)
print(f'The sample includes {n_sampled_phantoms} phantom cards.')

# for comparison audit
cards_to_retrieve, sample_order, cvr_sample, mvr_phantoms_sample = \
    Dominion.sample_from_cvrs(cvr_list, manifest, sampled_cvr_indices)

# for polling audit
# cards_to_retrieve, sample_order, mvr_phantoms_sample = Dominion.sample_from_manifest(manifest, sample)

# write the sample
# could write only the incremental sample using list(set(cards_to_retrieve) - set(cards_to_retrieve_old))
Dominion.write_cards_sampled(audit.sample_file, cards_to_retrieve, print_phantoms=False)

The sample includes 0 phantom cards.


In [58]:
# for real data
with open(audit.mvr_file) as f:
    mvr_json = json.load(f)

mvr_sample = CVR.from_dict(mvr_json['ballots'])

# for simulated data, no errors
mvr_sample = cvr_sample.copy()

## Find measured risks for all assertions

In [59]:
CVR.prep_comparison_sample(mvr_sample, cvr_sample, sample_order)  # for comparison audit
# CVR.prep_polling_sample(mvr_sample, sample_order)  # for polling audit

###### TEST
# permute part of the sample to introduce errors deliberately
mvr_sample = cvr_sample.copy()
n_errs = 5
errs = mvr_sample[0:n_errs].copy()
np.random.seed(12345678)
np.random.shuffle(errs)
mvr_sample[0:n_errs] = errs

In [60]:
p_max = Assertion.set_p_values(contests=contests, mvr_sample=mvr_sample, cvr_sample=cvr_sample)
print(f'maximum assertion p-value {p_max}')
done = audit.summarize_status(contests)

maximum assertion p-value 1.2395432394747009e-28

p-values for assertions in contest 339
	18 v 17 elim 15 16 45: 1.4449144036994632e-65
	17 v 16 elim 15 18 45: 1.2395432394747009e-28
	15 v 18 elim 16 17 45: 5.2774518365009846e-42
	18 v 16 elim 15 17 45: 6.205447790363212e-127
	17 v 16 elim 15 45: 2.8080650629218943e-86
	15 v 17 elim 16 45: 2.6490812247796027e-123
	15 v 17 elim 16 18 45: 1.7998086930305547e-166
	18 v 16 elim 15 45: 1.7022899132638306e-231
	15 v 16 elim 17 45: 4.74504262834718e-210
	15 v 16 elim 17 18 45: 2.445447527327654e-209
	15 v 16 elim 18 45: 9.191770721839916e-261
	15 v 16 elim 45: 8.342200191989171e-244
	15 v 45: 0.0

contest 339 AUDIT COMPLETE at risk limit 0.05. Attained risk 1.2395432394747009e-28
